<a href="https://colab.research.google.com/github/caccioppoli/etas_optimization/blob/main/etas_op.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import numpy as np
import pandas as pd
import pickle as pkl
import scipy.stats as stats
import matplotlib.pyplot as plt
from datetime import datetime
import time
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Loading instrumental catalog

In [82]:
with open("drive/MyDrive/Colab Notebooks/cat_italia.txt","r") as infile, open("drive/MyDrive/Colab Notebooks/cat_italia_conv.txt",'w') as outfile:
    for line in infile:
      outfile.write(" ".join(line.split()).replace('|',' '))
      outfile.write("\n")

# converto e memorizzo in secondi la data 

sdate = np.genfromtxt("drive/MyDrive/Colab Notebooks/cat_italia_conv.txt",dtype="str",delimiter=" ",usecols=(1))
startdate = datetime.strptime('1985-01-02T18:39:30.740000',"%Y-%m-%dT%H:%M:%S.%f")
seconds = []
for time in sdate:
  date = datetime.strptime(time,"%Y-%m-%dT%H:%M:%S.%f")
  temp = (date - startdate).total_seconds()
  seconds.append(temp)

# converto e memorizzo le altre grandezze del catalogo

latitude = np.genfromtxt("drive/MyDrive/Colab Notebooks/cat_italia_conv.txt",dtype="float",delimiter=" ",usecols=(2))
longitude = np.genfromtxt("drive/MyDrive/Colab Notebooks/cat_italia_conv.txt",dtype="float",delimiter=" ",usecols=(3))
depth = np.genfromtxt("drive/MyDrive/Colab Notebooks/cat_italia_conv.txt",dtype="float",delimiter=" ",usecols=(4))
magnitude = np.genfromtxt("drive/MyDrive/Colab Notebooks/cat_italia_conv.txt",dtype="float",delimiter=" ",usecols=(10))


0.0 48.092 7.061 3.7
